# Ananke

This notebook is based on the example notebook in py-ananke's repository "https://github.com/athob/py-ananke/tree/main"


In [ ]:
import ananke

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import h5py

In [ ]:
from astropy import coordinates as ac
from astropy import units as u

In [ ]:
idx_f = 178

In [ ]:
snapname = f"/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/out/snapshot_{idx_f - 1}.hdf5"

In [ ]:
starsname = "/astro/dboyea/sculptor/isolation/1e6/fiducial/stars_ana/exp2d_rs0.10_stars.hdf5"

In [ ]:
with h5py.File(snapname) as h5f:
    pos = np.array(h5f["PartType1/Coordinates"])
    vel = np.array(h5f["PartType1/Velocities"])
    index =  np.array(h5f["PartType1/ParticleIDs"])

In [ ]:
with h5py.File(starsname) as h5f:
    stars_index = np.array(h5f["index"])
    probabilities = np.array(h5f["probability"])
stars_index, probabilities

In [ ]:
def invperm(a):
    b = np.arange(a.shape[0])
    b[a] = b.copy()
    return b

In [ ]:
mtot = 2.7e6
mstar = probabilities[invperm(np.argsort(index))] * mtot

In [ ]:
stars_index[invperm(np.argsort(index))] == index

In [ ]:
sum(mstar)

In [ ]:
N = len(mstar)
V0 = 207.4

sim_input = {
    "pos3": pos,
    "vel3": vel * V0,
    "mass": mstar,
    "age": np.random.normal(9.8, 0.1, N),
    "feh": np.random.normal(-2.3, 0.25, N),
    "parentid": index,
    
}

In [ ]:
vel.shape

In [ ]:
sim_input

In [ ]:
idxs = np.random.choice(N, p=mstar/np.sum(mstar), size=10000)

In [ ]:
gcs = ac.Galactocentric(x=pos[idxs, 0]*u.kpc, y=pos[idxs, 1]*u.kpc, z=pos[idxs, 2]*u.kpc);

In [ ]:
icrs = gcs.transform_to(ac.ICRS())

In [ ]:
V0 = 207 # km/s

In [ ]:
help(ananke.Ananke)

In [ ]:
# matches my GC coordinate definition
observer = {
    "pos3": [8.122, 0, 0.0208],
    "vel3": [ 12.9, 245.6, 7.78],
}

rshell = [0, 1000]

fsample = 1

photo_sys = 'padova/GAIA__DR2'

cmd_magnames = {'magnitude': 'G',
                'color_minuend': 'Gbp',
                'color_subtrahend': 'Grp'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

In [ ]:
name = "fiducial"

an = ananke.Ananke(sim_input, name, 
    fsample=fsample, observe=observer, rshell=rshell, photo_sys=photo_sys, cmd_magnames=cmd_magnames, **cmd_box)


In [ ]:
survey = an.run()

In [ ]:
survey

In [ ]:
observable = survey[~survey.dec.isna()]
observable

In [ ]:
plt.scatter(observable.ra.values, observable.dec.values, alpha=0.1, s=1)
plt.xlabel("ra / degree")
plt.ylabel("dec / degree")

In [ ]:
plt.scatter(observable.mura.values, observable.mudec.values, alpha=0.1, s=1)

In [ ]:
dd = 5
plt.scatter(observable.ra.values + np.random.normal(0, 0.02, len(observable)),
            observable.dec.values + np.random.normal(0, 0.02, len(observable)), alpha=0.1, s=1)

In [ ]:
plt.scatter(observable.gaia__dr2_gbp.values - observable.gaia__dr2_grp.values, observable.gaia__dr2_g.values, s=1, alpha=0.1)
plt.gca().invert_yaxis()
plt.xlabel("Bp - Rp")
plt.ylabel("G")

In [ ]:

plt.scatter(icrs.ra + np.random.normal(0, 0.02, len(icrs))*u.degree, icrs.dec + np.random.normal(0, 0.02, len(icrs))*u.degree, s=1, alpha=0.1)

In [ ]:
icrs.ra

In [ ]:
observable["pmra"] = observable.mura
observable["pmdec"] = observable.mudec

observable["pmra_error"] = observable.mura_Err
observable["pmdec_error"] = observable.mudec_Err

observable["G"] = observable.gaia__dr2_g
observable["G_err"] = observable.gaia__dr2_g_Err
observable["bp_rp"] = observable.gaia__dr2_gbp - observable.gaia__dr2_grp
observable["bp_rp_err"] = observable.gaia__dr2_gbp - observable.gaia__dr2_grp

In [ ]:
observable["distance"] = 10 * 10 ** (1/5 * observable.dmod) / 1e3

In [ ]:
df = observable[["ra", "dec", "pmra", "pmdec", "pmra_error", "pmdec_error", "G", "bp_rp", "distance"]].to_pandas_df()
df["weights"] = 1 / len(df)

In [ ]:
from astropy.io import fits
from astropy.table import Table

In [ ]:
tab = Table.from_pandas(df)


In [ ]:
tab

In [ ]:
outfile = "anake_test.fits"

hdu = fits.BinTableHDU(tab)
hdu.writeto(outfile, overwrite=True)

In [ ]:
pwd()